In [ ]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.demo_db")

spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.demo_db.sales (
    order_id INT,
    customer STRING,
    amount DOUBLE
) USING iceberg
""")

print("✅ Table 'nessie.demo_db.sales' created.")


In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Iceberg-Nessie-MinIO-Demo")
    # ===== Iceberg Catalog qua Nessie =====
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v1")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://silver_fddđf/")
    # ===== Cấu hình MinIO (S3-compatible) =====
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    # ===== Spark + Hadoop S3 connector =====
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

print("✅ SparkSession connected to Nessie + MinIO via Iceberg.")


In [ ]:
data = [
    (1, "Alice", 100.5),
    (2, "Bob", 230.75),
    (3, "Charlie", 315.2)
]
df = spark.createDataFrame(data, ["order_id", "customer", "amount"])

df.writeTo("nessie.demo_db.sales").append()

print("✅ Data written successfully!")


In [ ]:
spark.sql("SELECT * FROM nessie.demo_db.sales").show()


In [ ]:
import requests
resp = requests.get("http://nessie:19120/api/v1/config")
print(resp.status_code)
print(resp.json())


In [ ]:
import os
from pynessie import NessieClient
from pynessie.conf import build_config

# 1. Khai báo endpoint Nessie (container trong cùng network)
os.environ["NESSIE_ENDPOINT"] = "http://nessie:19120/api/v1"

# 2. Tạo đối tượng config từ environment
config = build_config()

# 3. Truyền config này vào NessieClient
client = NessieClient(config)

# 4. Kiểm tra kết nối
print("✅ Connected to Nessie")
print("Current default branch:", client.get_default_branch())


In [ ]:
data2 = [
    (4, "David", 420.0),
    (5, "Eva", 180.5)
]
spark.createDataFrame(data2, ["order_id", "customer", "amount"])\
    .writeTo("nessie.demo_db.sales").append()

spark.sql("SELECT * FROM nessie.demo_db.sales").show()


In [ ]:
entry = next(client.get_log("main"))
print(entry)
print(dir(entry))


In [ ]:
for log_entry in client.get_log("main"):
    print(f"Commit hash: {log_entry.__hash__} | Message: {log_entry.commit_meta.message}")


In [ ]:
client.create_branch("experiment", "main")
print("✅ Branch 'experiment' created from 'main'")


In [ ]:
# Tạo SparkSession trỏ tới branch 'experiment'
spark_branch = (
    SparkSession.builder.appName("NessieExperimentBranch")
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v1")
    .config("spark.sql.catalog.nessie.ref", "experiment")   # 👈 Chuyển branch
    .config("spark.sql.catalog.nessie.warehouse", "s3a://warehouse/")
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .getOrCreate()
)

# Ghi thêm dữ liệu chỉ trong branch 'experiment'
data3 = [(6, "Frank", 999.9)]
spark_branch.createDataFrame(data3, ["order_id", "customer", "amount"])\
    .writeTo("nessie.demo_db.sales").append()

print("✅ Wrote data to branch 'experiment'")


In [ ]:
spark.sql("SELECT * FROM nessie.demo_db.sales ORDER BY order_id").show()


In [ ]:
spark_branch.sql("SELECT * FROM nessie.demo_db.sales ORDER BY order_id").show()


In [ ]:
spark.sql("SELECT * FROM nessie.demo_db.sales.snapshots").show(truncate=False)

In [ ]:
snapshot_id = 1791065480393736880

query = f"""
SELECT * FROM nessie.demo_db.sales VERSION AS OF {snapshot_id}
"""
old_df = spark.sql(query)
old_df.show()


In [ ]:
entry = next(client.get_log("main"))
print(entry)
print(dir(entry))
